In [ ]:
# @title Modules 
# Mounting Drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive

# Keras modules
import keras
from keras.models import Sequential
from keras.layers import Dense, Convolution2D, Flatten, MaxPooling2D, ZeroPadding2D, Dropout, BatchNormalization, Activation, DepthwiseConv2D, SeparableConv2D
from keras import optimizers
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, CSVLogger, RemoteMonitor, TensorBoard, ReduceLROnPlateau, History, EarlyStopping
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
import keras.backend as K
from tqdm.keras import TqdmCallback

#General
import numpy as np
from scipy import signal
from scipy.io import wavfile
import matplotlib.pyplot as plt
import glob
import datetime
import os
import pickle
from scipy.stats import mode
import tarfile
from tqdm import tqdm
import datetime
import math

#auth.authenticate_user()
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()
#drive = GoogleDrive(gauth)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title Git Pull
%cd /content/drive/MyDrive/EPFL Account/ML Project 2/ml_project2
! git reset --hard 
! git pull

/content/drive/MyDrive/EPFL Account/ML Project 2/ml_project2
^C
Already up to date.


In [ ]:
# @title Import our modules
%cd /content/drive/MyDrive/EPFL Account/ML Project 2/ml_project2/src
# specify the githum repo location as .../ml_project2/src

#Our Modules
import sound_processing as sp
from utils import*

/content/drive/MyDrive/EPFL Account/ML Project 2/ml_project2/src


In [ ]:
# @title Check the files
%cd /content/drive/MyDrive/ML Project 2/ml_project2/src
!ls -l ./npz_files/*/

[Errno 2] No such file or directory: '/content/drive/MyDrive/ML Project 2/ml_project2/src'
/content/drive/MyDrive/EPFL Account/ML Project 2/ml_project2/src
./npz_files/GVA/:
total 3079312
-rw------- 1 root root  553904757 Nov 27 16:23 GVA_Ca_Co_test.npz
-rw------- 1 root root 1369343627 Nov 27 16:16 GVA_Ca_train_b1.npz
-rw------- 1 root root 1229965595 Nov 27 16:13 GVA_Co_train_b1.npz

./npz_files/POA/:
total 21056243
-rw------- 1 root root  546407225 Nov 28 07:42 POA_Ca_Co_test.npz
-rw------- 1 root root 4121620181 Nov 27 15:29 POA_Ca_train_b1.npz
-rw------- 1 root root 4301549167 Nov 27 15:17 POA_Ca_train_b2.npz
-rw------- 1 root root 4519848135 Nov 27 15:41 POA_Ca_train_b3.npz
-rw------- 1 root root 4150727343 Nov 27 15:52 POA_Ca_train_b4.npz
-rw------- 1 root root 3088366560 Nov 27 16:07 POA_Co_train_b1.npz
-rw------- 1 root root  833071644 Nov 27 15:57 POA_Co_train_b2.npz


In [ ]:
# @title Generator
def generator(x_train, y_train, x_test, y_test):

  #Generates augmented data, we don't use it.

  train_batch_size = 32
  train_steps = x_train.shape[0]//train_batch_size # need to specify the number of steps since the data generator outputs continuously



  train_generator = ImageDataGenerator(width_shift_range=10,
                                       samplewise_center=False, 
                                       samplewise_std_normalization=False,
                                       featurewise_center=False,
                                       featurewise_std_normalization=False,
                                       #zoom_range=[1/1.3, 1.3],
                              #height_shift_range=4,
                              )

  test_generator = ImageDataGenerator(samplewise_center=False, 
                                       samplewise_std_normalization=False,
                                      featurewise_center=False,
                                      featurewise_std_normalization=False)

  train_generator.fit(x_train) #apply the augmetnation to train data
  test_generator.fit(x_test)
  train_datagen = train_generator.flow(x_train, y_train, batch_size=train_batch_size) # creates the mii batch flow
  test_datagen = test_generator.flow(x_test, y_test, batch_size=train_batch_size) 

  return train_datagen, test_datagen

In [ ]:
# @title Plotters
def add_subplot(history, fig, size, i, *name):
    print('Adding plot')
    Title = 'Run' + str(i) 
    ax = fig.add_subplot(math.ceil(size/2), 2, i)
    ax.plot(history.history['accuracy'])
    ax.plot(history.history['val_accuracy'])
    ax.plot(history.history['loss'])
    ax.plot(history.history['val_loss'])
    ax.set_title(Title)
    ax.set_ylabel('accuracy / loss')
    ax.set_xlabel('epoch')
    ax.legend(['train_acc', 'test_acc', 'train_loss', 'test_loss'], loc='lower left')
    ax.set_ylim(ymax=1, ymin=0)
    #plt.savefig(name + '.png')

def Plot_history(history , name, title='model accuracy / loss'):
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title(title)
  plt.ylabel('accuracy / loss')
  plt.xlabel('epoch')
  plt.legend(['train_acc', 'test_acc','train_loss','test_loss'], loc='lower left')
  plt.ylim(ymax = 1, ymin = 0)
  plt.savefig(name + '.png')
  plt.show()
    

In [ ]:
# @title Model
def def_model(shape):
  dense = 25
  model = Sequential()


  #model.add(Convolution2D(5, (1, 1), activation='relu', padding='same', input_shape=shape, data_format='channels_last') )
  #model.add(BatchNormalization())
  #model.add(MaxPooling2D(2, 2))
  #model.add(Dropout(0.25))

  model.add(Convolution2D(15, (3, 3), activation='relu', padding='same', input_shape=shape, data_format='channels_last') )
  model.add(BatchNormalization())
  model.add(MaxPooling2D(2, 2))
  model.add(Dropout(0.25))



  model.add(Convolution2D(30, (3, 3), activation='relu', padding='same', input_shape=shape, data_format='channels_last'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(2, 2))
  model.add(Dropout(0.25))

  model.add(Convolution2D(5, (1, 1), activation='relu', padding='same', input_shape=shape, data_format='channels_last') )
  model.add(BatchNormalization())
  model.add(MaxPooling2D(2, 2))
  model.add(Dropout(0.25))


  model.add(Flatten())
  model.add(Dense(dense, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))

  model.add(Dense(2, activation='softmax'))
  
  return model
  


In [ ]:
# @title Callbacks
def callbacks(run=''):
  """Created callbacks for the model. Early stopper, save best model only,
  log results, reduce learning rate."""
  time = datetime.datetime.now().strftime("-%Y-%m-%d") 

  dense = 25
  checkpointer = ModelCheckpoint(monitor='val_loss', # automatically saves the model 
                               mode='auto', 
                               filepath= './models/TRAIN_ON_POA_BATCH_1_RUN_' + str(run)+'_' + time + '.h5',
                               verbose=0,
                               save_best_only=True,
                               save_freq = 'epoch'
                              )
  csv_logger = CSVLogger('./models/TRAIN_ON_POA_BATCH_1_RUN_' + str(run) + '_' + time + '.txt') # saves the output log in csv

  lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, # can be used to reduce the learning rate once the model stops learning
                       min_delta=0.001, cooldown=1,
                       verbose=0, patience=10,
                        min_lr=4.0e-7)
  early = EarlyStopping(monitor='val_loss',
                        min_delta=0.002,
                        patience=20,
                        verbose=0,
                        mode='auto', 
                        baseline=None,
                        restore_best_weights=False
)


  return checkpointer, csv_logger, lr, early


In [ ]:
# @title Train Model
def train(x_train, y_train, x_test, y_test, run=''):
  """Trains model and returns the history object"""
  

  #train_datagen, test_datagen = generator(x_train, y_train, x_test, y_test)
  shape = x_train.shape[1:]



  model = def_model(shape) # load the defined model
  #model = keras.models.load_model('./models/TRAIN_ON_POA_BATCH_1/TRAIN_ON_POA_BATCH_1_RUN_7_-2020-12-12.h5', custom_objects={'f1': f1})
  # loads the pretrained model for transfer learning

  sgd = optimizers.SGD(lr=0.00004, decay=1e-6, momentum=0.9, nesterov=True)
  # define gradient descent 

  model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy', f1])
  # compile model

  cb1, cb2, cb3, cb4 = callbacks(run)
   
  train_batch_size = 32
  train_steps = x_train.shape[0]//train_batch_size # need to specify the number of steps since the data generator outputs continuously
  #FitTheModel

  history = model.fit(x_train, y_train,
                      steps_per_epoch=train_steps,
                      epochs=200,
                      validation_data=(x_test, y_test),
                      callbacks=[cb1, cb2, cb3, cb4, TqdmCallback(verbose=0)],
                      verbose=0
                      )
  
  max_val_acc = max(history.history['val_accuracy'])
  min_val_loss = min(history.history['val_loss'])
  max_val_f1 = max(history.history['val_f1'])
  print('Min val_loss = {}, Max val_acc = {}, Max val_f1 = {}'.format(min_val_loss, max_val_acc, max_val_f1))
  
  return history


In [ ]:
def train_and_plot():

  stat = []
  fig = plt.figure(figsize=(15, 30))
  N = 10
  
  x_train, y_train, x_test, y_test = get_arrays_POA(batch=1)
  #for GVA replace the POA(batch=1) with GVA(), for other batches replace 1 
  #with the batch number

  print(x_train.shape)

  for n in range(1, N):
    print('Run' + str(n))
    history = train(x_train, y_train, x_test, y_test, str(n))
    #Plot_history(history, 'stft of all positions stacked', title='stft of all positions stacked model accuracy_small_4_1DFilter / loss')
    stat.append((min(history.history['val_loss']), max(history.history['val_accuracy']), max(history.history['val_f1'])))

    plt.suptitle('Trained on POA Batch 1')
    #name = "Intermediate_stft_3"
    add_subplot(history, fig, N, n)
  
  #plt.savefig('TRAIN_ON_POA_BATCH_1.pdf')                              
  plt.show()

  for i in np.arange(len(stat)):
    plt.bar(i+1 + -0.25, stat[i][0], color = 'r', width = 0.25)
    plt.bar(i+1 + 0., stat[i][1], color = 'b', width = 0.25)
    plt.bar(i+1 + 0.25, stat[i][2], color = 'gold', width = 0.25)

  plt.title('Trained on POA Batch 1')
  plt.legend(['test_loss', 'test_acc','test_f1'], loc='lower left')
  plt.xlabel('Run')
  plt.ylim(top=1)
  #plt.savefig('TRAIN_ON_POA_BATCH_1_HIST.pdf')
  plt.show()

  

train_and_plot()



In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
def load_and_predict_GVA():
  print('Loading the model')
  model = keras.models.load_model('./models/TRAIN_ON_GVA_REM_POS_9/TRAIN_ON_GVA_TFINAL_RUN_3_-2020-12-12.h5', custom_objects={'f1': f1}) # best model preloaded
  print('Loading data for GVA') 
  x_train, y_train, _, _ = get_arrays_GVA()
  print('Done creating arrays for GVA ')
  sgd = optimizers.SGD(lr=0.00004, decay=1e-6, momentum=0.9, nesterov=True)
  model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy', f1])
  print('Predicting on GVA ')
  score = model.evaluate(x_train, y_train, batch_size = 32)
  for i in range(3):
    print(model.metrics_names[i],  '%.4f' %  score[i])
load_and_predict_GVA()

In [ ]:
def load_and_predict_POA(batch):
  """Load the pretrained model and predict on unseen data from POA"""
  print('Loading the model')
  model = keras.models.load_model('./models/TRAIN_ON_POA_BATCH_1/TRAIN_ON_POA_BATCH_1_RUN_7_-2020-12-12.h5', custom_objects={'f1': f1}) #  best model preloaded
  print('Loading data for batch ' + str(batch))
  x_train, y_train, _, _ = get_arrays_POA(batch)
  print('Done creating arrays for batch ' + str(batch))
  sgd = optimizers.SGD(lr=0.00004, decay=1e-6, momentum=0.9, nesterov=True)
  model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy', f1])
  print('Predicting on batch ' + str(batch))
  score = model.evaluate(x_train, y_train, batch_size = 32)
  for i in range(3):
    print(model.metrics_names[i],  '%.4f' %  score[i])

load_and_predict_POA(2)

In [ ]:
def print_stat():
  """Loads the model logs, prints average statistics and the best model num"""
  full_path  = 'models/TRAIN_ON_POA_BATCH_2' #specify the path of logs


  val_loss = []
  val_acc = []  
  val_f1 = []
  for file in os.listdir(full_path):
    if file.endswith(".txt"):
      acc, f1, loss = np.loadtxt(os.path.join(full_path, file), delimiter=',', skiprows=1, usecols=(4, 5, 6), unpack=True)
      val_loss.append(np.min(loss))
      val_acc.append(acc[np.argmin(loss)])
      val_f1.append(f1[np.argmin(loss)])
  val_loss = np.array(val_loss)
  val_acc = np.array(val_acc)
  val_f1 = np.array(val_f1)


  print('Acc: Mean {}, STD: {}'.format(np.mean(val_acc), np.std(val_acc)))   
  print('F1: Mean {}, STD: {}'.format(np.mean(val_f1), np.std(val_f1)))   
  print('Loss: Mean {}, STD: {}'.format(np.mean(val_loss), np.std(val_loss)))   

  print('Best Model by loss {}, Loss: {}, Acc: {}, F1: {}'.format(np.argmin(val_loss) + 1, np.min(val_loss), val_acc[np.argmin(val_loss)], val_f1[np.argmin(val_loss)]))
  print('Best Model by Acc {}, Loss: {}, Acc: {}, F1: {}'.format(np.argmax(val_acc) + 1, val_loss[np.argmax(val_acc)], np.max(val_acc), val_acc[np.argmax(val_acc)] ))
  print('Best Model by F1 {}, Loss: {}, Acc: {}, F1: {}'.format(np.argmax(val_f1) + 1, val_loss[np.argmax(val_f1)], val_acc[np.argmax(val_f1)], np.max(val_f1)))

print_stat()

Acc: Mean 0.878000009059906, STD: 0.06823488660602767
F1: Mean 0.8882812023162842, STD: 0.08421934061250606
Loss: Mean 0.33901224583387374, STD: 0.10429474055107064
Best Model by loss 9, Loss: 0.20125041902065277, Acc: 0.9700000286102295, F1: 0.9765625
Best Model by Acc 9, Loss: 0.20125041902065277, Acc: 0.9700000286102295, F1: 0.9700000286102295
Best Model by F1 9, Loss: 0.20125041902065277, Acc: 0.9700000286102295, F1: 0.9765625
